In [4]:
# Basic decorator (no arguments)

def logger(func):
    def wrapper(*args, **kwargs):
        print(f"calling {func.__name__} with {args} {kwargs}")
        result = func(*args, **kwargs)
        print(f"Result: {result}")
        return result
    return wrapper

@logger
def add(a, b):
    return a + b

add(3, b=7)

calling add with (3,) {'b': 7}
Result: 10


10

In [5]:
# 2. Decorator with arguments
def repeat(n):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for i in range(n):
                func(*args, **kwargs)
        return wrapper
    return decorator

@repeat(3)
def hello():
    print("Hello!")

hello()

Hello!
Hello!
Hello!


In [1]:
# 3. Decorator preserving function metadata (functools.wraps)

from functools import wraps

def uppercase(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper()
    return wrapper

@uppercase
def greet(name):
    return f"hello, {name}"

print(greet("Mohan"))
print(greet.__name__)
print(greet.__doc__)

HELLO, MOHAN
greet
None


In [6]:
# 4. Multiple decorators applied

def exlaim(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs) + "!"
    return wrapper

def question(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs) + "?"
    return wrapper

@exlaim
@question
def greet():
    return "Hello"

print(greet())

Hello?!


In [7]:
# 5. Decorator with both positional and keyword arguments

def debug(func):
    def wrapper(*args, **kwargs):
        print(f"Args: {args}, kwargs: {kwargs}")
        return func(*args, **kwargs)
    return wrapper

@debug
def divide(a, b=1):
    return a/b

print(divide(10, b=2))

Args: (10,), kwargs: {'b': 2}
5.0


In [8]:
# 1. Basic Class-based Decorator

class logger:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        print(f"starting '{self.func.__name__}'...")
        result = self.func(*args, **kwargs)
        print(f"Finished '{self.func.__name__}'.")
        return result
    
@logger
def greet(name):
    print(f"Hello, {name}")

greet("Mohan")

starting 'greet'...
Hello, Mohan
Finished 'greet'.


In [9]:
# 2. Class-based Decorator with Arguments

class prefixlogger:
    def __init__(self, prefix):
        self.prefix = prefix

    def __call__(self, func):
        def wrapper(*args, **kwargs):
            print(f"{self.prefix} - Executing {func.__name__}")
            return func(*args, **kwargs)
        return wrapper
    
@prefixlogger("DEBUG")
def add(a, b):
    return a + b

print(add(5, 7))

DEBUG - Executing add
12


In [10]:
# 3. Caching with Class-based Decorator

class cache:
    def __init__(self, func):
        self.func = func
        self.cache = {}

    def __call__(self, *args):
        if args in self.cache:
            print("Fetching from cache...")
            return self.cache[args]
        print("Computing result...")
        result = self.func(*args)
        self.cache[args] = result
        return result
    
@cache
def multiply(a, b):
    return a * b

print(multiply(3, 4))
print(multiply(3, 4))

Computing result...
12
Fetching from cache...
12


In [19]:
# 4. Retry Logic Decorator

import random

class retry:
    def __init__(self, retries):
        self.retries  =  retries

    def __call__(self, func):
        def wrapper(*args, **kwargs):
            for attempt in range(1, self.retries + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"Attempt {attempt} failed: {e}")
            raise RuntimeError("All retries failed")
        return wrapper


@retry(3)
def unstabletask():
    if random.random() < 0.7:
        raise ValueError("Random failure")
    return "Success!"

print(unstabletask())

Attempt 1 failed: Random failure
Attempt 2 failed: Random failure
Success!


In [21]:
# 5. Timing Function Execution

import time

class timer:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        start = time.time()
        result = self.func(*args, **kwargs)
        end = time.time()
        print(f"{self.func.__name__} took {end - start:.4f} seconds")
        return result
    
@timer
def slowfunction():
    time.sleep(1)

    return "Done!"

slowfunction()

slowfunction took 1.0152 seconds


'Done!'

In [24]:
# 6. Argument Validation

class postiveargs:
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args):
        if not all(isinstance(a, int) and a > 0 for a in args):
            raise ValueError("All arguments must be positive integers")
        return self.func(*args)
    


@postiveargs
def add(a, b):
    return a + b

print(add(3, 5))

8


In [29]:
# 7. Access Control Decorator

class requireadmin:
    def __init__(self, func):
        self.func = func

    def __call__(self, userrole, *args, **kwargs):
        if userrole != "admin":
            raise PermissionError("User must be an admin")
        return self.func(userrole, *args, **kwargs)
    
@requireadmin
def deleteuser(role, username):
    print(f"User {username} deleted.")

deleteuser("admin", "John doe")
deleteuser("user", "John doe")

User John doe deleted.


PermissionError: User must be an admin

In [31]:
# 8. Class-based Decorator with State Tracking

class callcounter:
    def __init__(self, func):
        self.func = func
        self.count = 0

    def __call__(self, *args, **kwargs):
        self.count += 1
        print(f"{self.func.__name__} has been called {self.count} times")
        return self.func(*args, **kwargs)
    
@callcounter
def greet(name):
    print(f"Hello, {name}")

greet("Mohan")
greet("Balor")

greet has been called 1 times
Hello, Mohan
greet has been called 2 times
Hello, Balor


In [32]:
# 9. Parameterized Class Decorator for Rate Limiting

import time

class ratelimit:
    def __init__(self, seconds):
        self.seconds = seconds
        self.lastcalled = 0
        
    def __call__(self, func):
        def wrapper(*args, **kwargs):
            now = time.time()
            if now - self.lastcalled < self.seconds:
                raise RuntimeError("Too many calls")
            self.lastcalled = now
            return func(*args, **kwargs)
        return wrapper
    
@ratelimit(2)
def ping():
    print("Ping!")

ping()


Ping!


In [ ]:
# 10. Stacking Multiple Class-based Decorators

class uppercase:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs).upper()
    
class exclaim:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs) + "!!!"
    
@exclaim
@uppercase
def greet(name):
    return f"Hello, {name}"

greet("alice")

'HELLO, ALICE'